In [1]:
1+2

3

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Coal,Part,Repart").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/28 21:38:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df1 = spark.read.csv("DataSets/transactions.csv",header=True)
df1.show(5)

+-----------+-------------+--------+-------------------+----------------+-------+
|Customer_No|    Card_type|    Date|           Category|Transaction Type| Amount|
+-----------+-------------+--------+-------------------+----------------+-------+
|    1000501|Platinum Card|1/1/2018|           Shopping|           debit|  11.11|
|    1000501|     Checking|1/2/2018|    Mortgage & Rent|           debit|1247.44|
|    1000501|  Silver Card|1/2/2018|        Restaurants|           debit|  24.22|
|    1000501|Platinum Card|1/3/2018|Credit Card Payment|          credit|2298.09|
|    1000501|Platinum Card|1/4/2018|      Movies & DVDs|           debit|  11.76|
+-----------+-------------+--------+-------------------+----------------+-------+
only showing top 5 rows



In [5]:
df1.rdd.getNumPartitions()

1

In [9]:
df1.count()

105

In [10]:
df1.select("Customer_No").distinct().show()

+-----------+
|Customer_No|
+-----------+
|    1001863|
|    1001368|
|    1000210|
|    1000531|
|    1000654|
|    1002324|
|    1000501|
+-----------+



## repartition

with this we can increase, decrease the number of partitions.

In [ ]:
df1.repartition(4).rdd.getNumPartitions()

In [13]:
df1.repartition("Customer_No").rdd.getNumPartitions()
# It can done repartition based on spark optimal number of partitions only

1

In [16]:
df1.repartition(3,"Customer_No").rdd.getNumPartitions()

# here I am giving 3 as number of partitions

3

In [19]:
from pyspark.sql.functions import spark_partition_id

In [20]:
df1.repartition(3,"Customer_No").withColumn("Partition ID",spark_partition_id()).groupBy("Partition ID").count().show()

+------------+-----+
|Partition ID|count|
+------------+-----+
|           0|   31|
|           1|   14|
|           2|   60|
+------------+-----+



## coalesce

with this we can only decrease the number of partitions.

In [22]:
df1.repartition(4).coalesce(2).rdd.getNumPartitions()

2

In [23]:
df1.repartition(4).coalesce(6).rdd.getNumPartitions()

4

## partitionBy

In [24]:
df1.write.partitionBy("Customer_No")

In [28]:
df1.write.partitionBy("Customer_No").format("csv").option("header","true").save("Hello_Transactions")

# Important points:

repartition:
It store the data in memory, it involves in shuffle



partitionBy:
It store the data directly in disc, no shuffle

In most of the cases, coalesce() does not trigger a shuffle. The coalesce() can be used soon after heavy filtering to optimize the execution time. It is important to notice that coalesce() does not always avoid shuffling.


partitionBy is for writing the df into separate subdirectories based on partition column, and writes directly to the disc.

While repartition and colasce deal with distribution of data inmemory among executors. 

coalesce is used to decrease the partitions and it may avoid full reshuffling, so it will be faster than repartition.

repartition can both increase or decrease partitions but it does with full reshuffle. 

But if you decrease the partition too much than capacity of executor, it can lead to OOM(Out Of Memory issue). 

So it depends on the problem which we are solving ie if we want to reduce skewness or distribute more.
